# Dataset: Israel's Covid-19 Dataset for Year 2020 & 2021
Link to a dataset: https://www.kaggle.com/datasets/mykeysid10/covid19-dataset-for-year-2020

In [4]:
import numpy as np
import pandas as pd

from sklearn.metrics import make_scorer, accuracy_score, f1_score

from sklearn import model_selection
from sklearn.model_selection import (
    StratifiedKFold,
    cross_validate,
    GridSearchCV,
)

Standard models used:

In [5]:
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

**fcalc** library files are located in **FCALC** folder

**FCALC** folder is located in the same directory as this notebook.

In [3]:
import FCALC.fcalc as fcalc

# Loading data

In [10]:
data_isr = pd.read_csv('Datasets/covid_israel.csv', 
                       sep=',',
                       true_values=['Yes','yes', 'Positive'],
                       false_values=['No','no', 'Negative']
                      )
data_isr.dropna(inplace = True)
data_isr.drop_duplicates(inplace=True)

data_isr.head()

,Unnamed: 0,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,test_indication
0,0,0,0,0,0,0,False,True,Other
1,2,0,0,0,0,0,False,False,Other
2,218,0,0,0,0,0,True,False,Other
3,234,0,0,0,0,0,True,True,Other
4,239,0,0,0,0,0,True,False,Contact with confirmed


In [11]:
onehot = pd.get_dummies(data_isr.test_indication)
onehot.drop(['Other'], axis=1, inplace=True)

df = pd.concat([onehot, data_isr], axis=1)
df.drop(['test_indication'], axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)
df = df.astype(bool)

# changing the order of columns
new_order =[
     'Abroad','Contact with confirmed',
     'cough', 'fever', 'sore_throat',
     'shortness_of_breath', 'head_ache', 
     'age_60_and_above', 'corona_result'
]
df = df[new_order]

df

,Abroad,Contact with confirmed,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,corona_result
0,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,True,True
4,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...
361,True,False,False,True,False,True,True,True,True
362,True,False,True,True,False,True,True,True,False
363,True,False,False,False,True,True,True,False,True
364,True,False,False,True,True,True,True,False,True


In [12]:
df.shape

(366, 9)

In [82]:
df.corona_result.value_counts(normalize=True)

corona_result
False    0.5
True     0.5
Name: proportion, dtype: float64

In [83]:
features = df.columns[0:-1]
target = df.columns[-1]

In [84]:
X = df[features]
y = df[target]

# Testing models

In [85]:
scoring = {'accuracy' : make_scorer(accuracy_score),
           'f1_macro' : make_scorer(f1_score, average='macro'),
           'f1_binary' : make_scorer(f1_score),
          }

kfold = StratifiedKFold(n_splits=10, random_state=49, shuffle=True)

In [86]:
def print_results(results):
    acc = np.round(np.mean(results['test_accuracy']), 4) 
    f1_m = np.round(np.mean(results['test_f1_macro']), 4)
    f1_b = np.round(np.mean(results['test_f1_binary']), 4)
    print(f'Accuracy = {acc:0.4f}, F1_macro = {f1_m:0.4f}, F1_binary = {f1_b:0.4f}')  

# Standard models

### Logistic regression

In [87]:
parameters = {'C' : np.linspace(1,5,101)}

model = LogisticRegression()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 101 candidates, totalling 1010 fits


{'C': 1.88}

In [88]:
model = LogisticRegression(C=best.best_params_['C'])
results = cross_validate(estimator=model,
                         X=X, y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.3629, F1_macro = 0.3595, F1_binary = 0.3485


# K-Nearest Neighbours

In [89]:
parameters = {'n_neighbors' : range(5,78,4)}

model = KNeighborsClassifier()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 19 candidates, totalling 190 fits


{'n_neighbors': 45}

In [90]:
model = KNeighborsClassifier(n_neighbors=best.best_params_['n_neighbors'])
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.4751, F1_macro = 0.4367, F1_binary = 0.5811


# Naive Bayes

##### MULTINOMIAL NB

In [91]:
parameters = {'alpha' : np.linspace(1,5,901)}

model = MultinomialNB()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 901 candidates, totalling 9010 fits


{'alpha': 1.0}

In [92]:
model = MultinomialNB(alpha=best.best_params_['alpha'])

results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.3522, F1_macro = 0.3467, F1_binary = 0.3490


##### GAUSSIAN NB

In [93]:
model = GaussianNB()
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.3633, F1_macro = 0.3609, F1_binary = 0.3643


##### COMPLEMENT NB

In [94]:
parameters = {'alpha' : np.linspace(1,12,111)}

model =  ComplementNB()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 111 candidates, totalling 1110 fits


{'alpha': 8.600000000000001}

In [95]:
model = ComplementNB(alpha=best.best_params_['alpha'])
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results) 

Accuracy = 0.3577, F1_macro = 0.3546, F1_binary = 0.3495


### Decision tree

In [101]:
parameters = {'min_samples_split' : range(2,23,2),
              'max_depth' : range(2,21,2),
              'criterion' : ['gini', 'entropy']
             }

model = DecisionTreeClassifier()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 220 candidates, totalling 2200 fits


{'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2}

In [102]:
model = DecisionTreeClassifier(
    max_depth=best.best_params_['max_depth'], 
    min_samples_split=best.best_params_['min_samples_split'],
    criterion=best.best_params_['criterion']
)
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.3493, F1_macro = 0.3343, F1_binary = 0.2666


### RandomForest

In [103]:
parameters = {'n_estimators' : range(40,120,10),
              'min_samples_split' : range(2,13,2),
              'max_depth' : range(2,17,2),
              'criterion' : ['gini', 'entropy']
             }
model = RandomForestClassifier()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=3, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 768 candidates, totalling 7680 fits
[CV 1/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=40; accuracy: (test=0.270) f1_binary: (test=0.341) f1_macro: (test=0.262) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=40; accuracy: (test=0.324) f1_binary: (test=0.242) f1_macro: (test=0.316) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=40; accuracy: (test=0.351) f1_binary: (test=0.520) f1_macro: (test=0.260) total time=   0.0s
[CV 4/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=40; accuracy: (test=0.297) f1_binary: (test=0.350) f1_macro: (test=0.293) total time=   0.0s
[CV 5/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=40; accuracy: (test=0.324) f1_binary: (test=0.138) f1_macro: (test=0.291) total time=   0.0s
[CV 6/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=40; accu

[CV 9/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=80; accuracy: (test=0.278) f1_binary: (test=0.188) f1_macro: (test=0.269) total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=80; accuracy: (test=0.222) f1_binary: (test=0.222) f1_macro: (test=0.222) total time=   0.0s
[CV 1/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=90; accuracy: (test=0.378) f1_binary: (test=0.439) f1_macro: (test=0.371) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=90; accuracy: (test=0.351) f1_binary: (test=0.333) f1_macro: (test=0.351) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=90; accuracy: (test=0.405) f1_binary: (test=0.421) f1_macro: (test=0.405) total time=   0.0s
[CV 4/10] END criterion=gini, max_depth=2, min_samples_split=2, n_estimators=90; accuracy: (test=0.216) f1_binary: (test=0.216) f1_macro: (test=0.216

[CV 8/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=50; accuracy: (test=0.333) f1_binary: (test=0.250) f1_macro: (test=0.325) total time=   0.0s
[CV 9/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=50; accuracy: (test=0.444) f1_binary: (test=0.444) f1_macro: (test=0.444) total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=50; accuracy: (test=0.278) f1_binary: (test=0.316) f1_macro: (test=0.276) total time=   0.0s
[CV 1/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=60; accuracy: (test=0.378) f1_binary: (test=0.343) f1_macro: (test=0.377) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=60; accuracy: (test=0.378) f1_binary: (test=0.489) f1_macro: (test=0.348) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=60; accuracy: (test=0.432) f1_binary: (test=0.533) f1_macro: (test=0.405

[CV 7/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=100; accuracy: (test=0.306) f1_binary: (test=0.324) f1_macro: (test=0.305) total time=   0.0s
[CV 8/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=100; accuracy: (test=0.417) f1_binary: (test=0.087) f1_macro: (test=0.329) total time=   0.0s
[CV 9/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=100; accuracy: (test=0.222) f1_binary: (test=0.222) f1_macro: (test=0.222) total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=100; accuracy: (test=0.222) f1_binary: (test=0.263) f1_macro: (test=0.220) total time=   0.0s
[CV 1/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=110; accuracy: (test=0.324) f1_binary: (test=0.419) f1_macro: (test=0.306) total time=   0.1s
[CV 2/10] END criterion=gini, max_depth=2, min_samples_split=4, n_estimators=110; accuracy: (test=0.405) f1_binary: (test=0.476) f1_macro: (test

[CV 7/10] END criterion=gini, max_depth=2, min_samples_split=6, n_estimators=70; accuracy: (test=0.361) f1_binary: (test=0.303) f1_macro: (test=0.357) total time=   0.0s
[CV 8/10] END criterion=gini, max_depth=2, min_samples_split=6, n_estimators=70; accuracy: (test=0.333) f1_binary: (test=0.400) f1_macro: (test=0.325) total time=   0.0s
[CV 9/10] END criterion=gini, max_depth=2, min_samples_split=6, n_estimators=70; accuracy: (test=0.250) f1_binary: (test=0.182) f1_macro: (test=0.245) total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=2, min_samples_split=6, n_estimators=70; accuracy: (test=0.250) f1_binary: (test=0.229) f1_macro: (test=0.249) total time=   0.0s
[CV 1/10] END criterion=gini, max_depth=2, min_samples_split=6, n_estimators=80; accuracy: (test=0.324) f1_binary: (test=0.419) f1_macro: (test=0.306) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=2, min_samples_split=6, n_estimators=80; accuracy: (test=0.351) f1_binary: (test=0.333) f1_macro: (test=0.351

...

[CV 10/10] END criterion=entropy, max_depth=16, min_samples_split=8, n_estimators=100; accuracy: (test=0.056) f1_binary: (test=0.056) f1_macro: (test=0.056) total time=   0.0s
[CV 1/10] END criterion=entropy, max_depth=16, min_samples_split=8, n_estimators=110; accuracy: (test=0.108) f1_binary: (test=0.057) f1_macro: (test=0.105) total time=   0.1s
[CV 2/10] END criterion=entropy, max_depth=16, min_samples_split=8, n_estimators=110; accuracy: (test=0.135) f1_binary: (test=0.059) f1_macro: (test=0.129) total time=   0.1s
[CV 3/10] END criterion=entropy, max_depth=16, min_samples_split=8, n_estimators=110; accuracy: (test=0.081) f1_binary: (test=0.150) f1_macro: (test=0.075) total time=   0.1s
[CV 4/10] END criterion=entropy, max_depth=16, min_samples_split=8, n_estimators=110; accuracy: (test=0.054) f1_binary: (test=0.054) f1_macro: (test=0.054) total time=   0.1s
[CV 5/10] END criterion=entropy, max_depth=16, min_samples_split=8, n_estimators=110; accuracy: (test=0.054) f1_binary: (tes

[CV 7/10] END criterion=entropy, max_depth=16, min_samples_split=10, n_estimators=70; accuracy: (test=0.139) f1_binary: (test=0.114) f1_macro: (test=0.138) total time=   0.0s
[CV 8/10] END criterion=entropy, max_depth=16, min_samples_split=10, n_estimators=70; accuracy: (test=0.111) f1_binary: (test=0.111) f1_macro: (test=0.111) total time=   0.0s
[CV 9/10] END criterion=entropy, max_depth=16, min_samples_split=10, n_estimators=70; accuracy: (test=0.083) f1_binary: (test=0.000) f1_macro: (test=0.077) total time=   0.0s
[CV 10/10] END criterion=entropy, max_depth=16, min_samples_split=10, n_estimators=70; accuracy: (test=0.167) f1_binary: (test=0.118) f1_macro: (test=0.164) total time=   0.0s
[CV 1/10] END criterion=entropy, max_depth=16, min_samples_split=10, n_estimators=80; accuracy: (test=0.081) f1_binary: (test=0.056) f1_macro: (test=0.080) total time=   0.0s
[CV 2/10] END criterion=entropy, max_depth=16, min_samples_split=10, n_estimators=80; accuracy: (test=0.108) f1_binary: (tes

[CV 5/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=40; accuracy: (test=0.135) f1_binary: (test=0.111) f1_macro: (test=0.135) total time=   0.0s
[CV 6/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=40; accuracy: (test=0.162) f1_binary: (test=0.061) f1_macro: (test=0.152) total time=   0.0s
[CV 7/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=40; accuracy: (test=0.139) f1_binary: (test=0.114) f1_macro: (test=0.138) total time=   0.0s
[CV 8/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=40; accuracy: (test=0.139) f1_binary: (test=0.162) f1_macro: (test=0.138) total time=   0.0s
[CV 9/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=40; accuracy: (test=0.111) f1_binary: (test=0.059) f1_macro: (test=0.108) total time=   0.0s
[CV 10/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=40; accuracy: (test=0.139) f1_binary: (tes

[CV 3/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=90; accuracy: (test=0.162) f1_binary: (test=0.244) f1_macro: (test=0.152) total time=   0.0s
[CV 4/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=90; accuracy: (test=0.108) f1_binary: (test=0.154) f1_macro: (test=0.105) total time=   0.0s
[CV 5/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=90; accuracy: (test=0.135) f1_binary: (test=0.158) f1_macro: (test=0.135) total time=   0.0s
[CV 6/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=90; accuracy: (test=0.162) f1_binary: (test=0.114) f1_macro: (test=0.160) total time=   0.0s
[CV 7/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=90; accuracy: (test=0.194) f1_binary: (test=0.216) f1_macro: (test=0.194) total time=   0.0s
[CV 8/10] END criterion=entropy, max_depth=16, min_samples_split=12, n_estimators=90; accuracy: (test=0.139) f1_binary: (test

{'criterion': 'gini',
 'max_depth': 2,
 'min_samples_split': 6,
 'n_estimators': 60}

In [104]:
model = RandomForestClassifier(
    n_estimators=best.best_params_['n_estimators'],
    min_samples_split=best.best_params_['min_samples_split'],
    max_depth=best.best_params_['max_depth'],
    criterion=best.best_params_['criterion']
)
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.3335, F1_macro = 0.3099, F1_binary = 0.2816


# Lazy FCA

Due to the multilabel output of **BinarizedBinaryClassifier** (**1** for **True**, **0** for **false** and **-1** for **undefined**) we cannot utilize **f1_score** with **average='binary'**. Therefore, a function that interprets **undefined** as misclassification was implemented.

In [107]:
def compare_with_binary_f1(y_true, y_pred):
    y_tmp = np.concatenate(
        (np.array(y_true)[:,None],np.array(y_pred)[:,None]),
        axis=1
    )
    df_tmp = pd.DataFrame(y_tmp, columns=['y_true','y_pred'])
    df_tmp.y_true = df_tmp.y_true.astype(bool)
    df_tmp['y_new']= ~df_tmp.loc[df_tmp.y_pred==-1]['y_true']
    df_tmp.loc[df_tmp.y_pred!=-1, 'y_new'] = df_tmp.loc[df_tmp.y_pred!=-1,'y_pred'].astype(bool) 
    df_tmp.y_new = df_tmp.y_new.astype(bool)
    return(f1_score(df_tmp.y_true, df_tmp.y_new))

In [108]:
def results_fca(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    accuracy = accuracy_score(y_true, y_pred)
    f1_binary = compare_with_binary_f1(y_true, y_pred)
    return (accuracy, f1, f1_binary)  

## BinarizedBinaryClassifier

**BinarizedBinaryClassifier** is not a **scikit** model, so **GridSearchCV** and **cross_validate** from **scikit** do not support it. Thus, we check desired parameters in **for**  loops.

In [114]:
parameters = {'alpha' : np.linspace(0, 1, 21),
              'method': ['standard','standard-support','ratio-support'],
             }

n = kfold.get_n_splits(X)

f1_best = 0
alpha_best = 0.
method_best = 'standard'

for method in parameters['method']:
    for alpha in parameters['alpha']:
        
        f1 = [0 for i in range(n)]
        for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
            bin_cls = fcalc.classifier.BinarizedBinaryClassifier(
                X.iloc[train_index].values, 
                y.iloc[train_index].to_numpy(),
                method=method,
                alpha=alpha
            )
            bin_cls.predict(X.iloc[test_index].values)
            f1[i] = f1_score(y.iloc[test_index], bin_cls.predictions, average='macro')
            print(f"[CV {i+1}/{n}] method={method}, alpha={alpha:0.2f}, f1_macro={f1[i]:0.4f}") 
                    
        f1_mean = np.mean(f1)
        print(f'f1_mean = {f1_mean:0.4f}\n')
        if(f1_mean > f1_best):
            f1_best = f1_mean
            alpha_best = alpha
            method_best = method
            
print(f"f1_best={f1_best:0.4f}, method={method_best}, alpha={alpha_best:0.2f}")

[CV 1/10] method=standard, alpha=0.00, f1_macro=0.0000
[CV 2/10] method=standard, alpha=0.00, f1_macro=0.0000
[CV 3/10] method=standard, alpha=0.00, f1_macro=0.0000
[CV 4/10] method=standard, alpha=0.00, f1_macro=0.0533
[CV 5/10] method=standard, alpha=0.00, f1_macro=0.0000
[CV 6/10] method=standard, alpha=0.00, f1_macro=0.0000
[CV 7/10] method=standard, alpha=0.00, f1_macro=0.0000
[CV 8/10] method=standard, alpha=0.00, f1_macro=0.0000
[CV 9/10] method=standard, alpha=0.00, f1_macro=0.0000
[CV 10/10] method=standard, alpha=0.00, f1_macro=0.0303
f1_mean = 0.0084

[CV 1/10] method=standard, alpha=0.05, f1_macro=0.1127
[CV 2/10] method=standard, alpha=0.05, f1_macro=0.1160
[CV 3/10] method=standard, alpha=0.05, f1_macro=0.2013
[CV 4/10] method=standard, alpha=0.05, f1_macro=0.1296
[CV 5/10] method=standard, alpha=0.05, f1_macro=0.0952
[CV 6/10] method=standard, alpha=0.05, f1_macro=0.1119
[CV 7/10] method=standard, alpha=0.05, f1_macro=0.1010
[CV 8/10] method=standard, alpha=0.05, f1_macr

[CV 1/10] method=standard, alpha=0.75, f1_macro=0.3079
[CV 2/10] method=standard, alpha=0.75, f1_macro=0.5281
[CV 3/10] method=standard, alpha=0.75, f1_macro=0.5180
[CV 4/10] method=standard, alpha=0.75, f1_macro=0.4559
[CV 5/10] method=standard, alpha=0.75, f1_macro=0.3833
[CV 6/10] method=standard, alpha=0.75, f1_macro=0.3479
[CV 7/10] method=standard, alpha=0.75, f1_macro=0.1844
[CV 8/10] method=standard, alpha=0.75, f1_macro=0.2292
[CV 9/10] method=standard, alpha=0.75, f1_macro=0.2113
[CV 10/10] method=standard, alpha=0.75, f1_macro=0.2851
f1_mean = 0.3451

[CV 1/10] method=standard, alpha=0.80, f1_macro=0.3079
[CV 2/10] method=standard, alpha=0.80, f1_macro=0.5281
[CV 3/10] method=standard, alpha=0.80, f1_macro=0.5180
[CV 4/10] method=standard, alpha=0.80, f1_macro=0.4559
[CV 5/10] method=standard, alpha=0.80, f1_macro=0.3833
[CV 6/10] method=standard, alpha=0.80, f1_macro=0.3479
[CV 7/10] method=standard, alpha=0.80, f1_macro=0.1844
[CV 8/10] method=standard, alpha=0.80, f1_macr

[CV 9/10] method=standard-support, alpha=0.35, f1_macro=0.1656
[CV 10/10] method=standard-support, alpha=0.35, f1_macro=0.1961
f1_mean = 0.2428

[CV 1/10] method=standard-support, alpha=0.40, f1_macro=0.2160
[CV 2/10] method=standard-support, alpha=0.40, f1_macro=0.3478
[CV 3/10] method=standard-support, alpha=0.40, f1_macro=0.2771
[CV 4/10] method=standard-support, alpha=0.40, f1_macro=0.2278
[CV 5/10] method=standard-support, alpha=0.40, f1_macro=0.1417
[CV 6/10] method=standard-support, alpha=0.40, f1_macro=0.2681
[CV 7/10] method=standard-support, alpha=0.40, f1_macro=0.2611
[CV 8/10] method=standard-support, alpha=0.40, f1_macro=0.3566
[CV 9/10] method=standard-support, alpha=0.40, f1_macro=0.1497
[CV 10/10] method=standard-support, alpha=0.40, f1_macro=0.1961
f1_mean = 0.2442

[CV 1/10] method=standard-support, alpha=0.45, f1_macro=0.2160
[CV 2/10] method=standard-support, alpha=0.45, f1_macro=0.3478
[CV 3/10] method=standard-support, alpha=0.45, f1_macro=0.2771
[CV 4/10] method=

[CV 1/10] method=ratio-support, alpha=0.00, f1_macro=0.2885
[CV 2/10] method=ratio-support, alpha=0.00, f1_macro=0.3478
[CV 3/10] method=ratio-support, alpha=0.00, f1_macro=0.3217
[CV 4/10] method=ratio-support, alpha=0.00, f1_macro=0.3273
[CV 5/10] method=ratio-support, alpha=0.00, f1_macro=0.3223
[CV 6/10] method=ratio-support, alpha=0.00, f1_macro=0.3471
[CV 7/10] method=ratio-support, alpha=0.00, f1_macro=0.3566
[CV 8/10] method=ratio-support, alpha=0.00, f1_macro=0.2340
[CV 9/10] method=ratio-support, alpha=0.00, f1_macro=0.3455
[CV 10/10] method=ratio-support, alpha=0.00, f1_macro=0.2138
f1_mean = 0.3104

[CV 1/10] method=ratio-support, alpha=0.05, f1_macro=0.2885
[CV 2/10] method=ratio-support, alpha=0.05, f1_macro=0.3478
[CV 3/10] method=ratio-support, alpha=0.05, f1_macro=0.3217
[CV 4/10] method=ratio-support, alpha=0.05, f1_macro=0.3273
[CV 5/10] method=ratio-support, alpha=0.05, f1_macro=0.3223
[CV 6/10] method=ratio-support, alpha=0.05, f1_macro=0.3471
[CV 7/10] method=rati

[CV 6/10] method=ratio-support, alpha=0.65, f1_macro=0.3471
[CV 7/10] method=ratio-support, alpha=0.65, f1_macro=0.3566
[CV 8/10] method=ratio-support, alpha=0.65, f1_macro=0.2340
[CV 9/10] method=ratio-support, alpha=0.65, f1_macro=0.3455
[CV 10/10] method=ratio-support, alpha=0.65, f1_macro=0.2138
f1_mean = 0.3104

[CV 1/10] method=ratio-support, alpha=0.70, f1_macro=0.2885
[CV 2/10] method=ratio-support, alpha=0.70, f1_macro=0.3478
[CV 3/10] method=ratio-support, alpha=0.70, f1_macro=0.3217
[CV 4/10] method=ratio-support, alpha=0.70, f1_macro=0.3273
[CV 5/10] method=ratio-support, alpha=0.70, f1_macro=0.3223
[CV 6/10] method=ratio-support, alpha=0.70, f1_macro=0.3710
[CV 7/10] method=ratio-support, alpha=0.70, f1_macro=0.3125
[CV 8/10] method=ratio-support, alpha=0.70, f1_macro=0.2340
[CV 9/10] method=ratio-support, alpha=0.70, f1_macro=0.3455
[CV 10/10] method=ratio-support, alpha=0.70, f1_macro=0.2138
f1_mean = 0.3084

[CV 1/10] method=ratio-support, alpha=0.75, f1_macro=0.2885
[C

In [115]:
f1 = [0 for i in range(n)]
accuracy = [0 for i in range(n)]
f1_binary = [0 for i in range(n)]

for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
    bin_cls = fcalc.classifier.BinarizedBinaryClassifier(
        X.iloc[train_index].values, 
        y.iloc[train_index].to_numpy(),
        method=method_best,
        alpha=alpha_best
    )
    bin_cls.predict(X.iloc[test_index].values)
    
    accuracy[i], f1[i], f1_binary[i] = results_fca(y.iloc[test_index], bin_cls.predictions)
    
    print(f"[CV {i+1}/{n}] method={method_best}, alpha={alpha_best:0.2f}", 
          f"accuracy={accuracy[i]:0.4f}, f1_macro={f1[i]:0.4f}, f1_binary={f1_binary[i]:0.4f}")

f1_mean = np.mean(f1)
accuracy_mean = np.mean(accuracy)
f1_binary_mean = np.mean(f1_binary)

print(f"\n accuracy={accuracy_mean:0.4f}, f1_macro={f1_mean:0.4f}, f1_binary={f1_binary_mean:0.4f}")

[CV 1/10], method=ratio-support, alpha=1.0 accuracy=0.3243, f1_macro=0.2095, f1_binary=0.4186
[CV 2/10], method=ratio-support, alpha=1.0 accuracy=0.3514, f1_macro=0.3509, f1_binary=0.3684
[CV 3/10], method=ratio-support, alpha=1.0 accuracy=0.3784, f1_macro=0.3616, f1_binary=0.4651
[CV 4/10], method=ratio-support, alpha=1.0 accuracy=0.5405, f1_macro=0.5351, f1_binary=0.5854
[CV 5/10], method=ratio-support, alpha=1.0 accuracy=0.5676, f1_macro=0.5067, f1_binary=0.6800
[CV 6/10], method=ratio-support, alpha=1.0 accuracy=0.4595, f1_macro=0.4591, f1_binary=0.4444
[CV 7/10], method=ratio-support, alpha=1.0 accuracy=0.5000, f1_macro=0.3100, f1_binary=0.6087
[CV 8/10], method=ratio-support, alpha=1.0 accuracy=0.3611, f1_macro=0.3566, f1_binary=0.4103
[CV 9/10], method=ratio-support, alpha=1.0 accuracy=0.4444, f1_macro=0.4286, f1_binary=0.5238
[CV 10/10], method=ratio-support, alpha=1.0 accuracy=0.4444, f1_macro=0.3050, f1_binary=0.4444

 accuracy=0.4372, f1_macro=0.3823, f1_binary=0.4949


## PatternBinaryClassifier

In [117]:
parameters = {'alpha' : np.linspace(0, 1, 21),
              'method': ['standard','standard-support','ratio-support'],
             }

n = kfold.get_n_splits(X)

f1_best_p = 0
alpha_best_p = 0.
method_best_p = 'standard'

for method in parameters['method']:
    for alpha in parameters['alpha']:
        
        f1 = [0 for i in range(n)]
        for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
            bin_cls = fcalc.classifier.PatternBinaryClassifier(
                X.iloc[train_index].values, 
                y.iloc[train_index].to_numpy(),
                method=method,
                alpha=alpha
            )
            bin_cls.predict(X.iloc[test_index].values)
            f1[i] = f1_score(y.iloc[test_index], bin_cls.predictions, average='macro')
            print(f"[CV {i+1}/{n}] method={method}, alpha={alpha:0.2f}, f1_macro={f1[i]:0.4f}") 
                    
        f1_mean = np.mean(f1)
        print(f'f1_mean = {f1_mean:0.4f}\n')
        if(f1_mean > f1_best_p):
            f1_best_p = f1_mean
            alpha_best_p = alpha
            method_best_p = method
            
print(f"f1_best={f1_best_p:0.4f}, method={method_best_p}, alpha={alpha_best_p:0.2f}")

[CV 1/10] method=standard, alpha=0.00, f1_macro=0.0804
[CV 2/10] method=standard, alpha=0.00, f1_macro=0.0185
[CV 3/10] method=standard, alpha=0.00, f1_macro=0.0804
[CV 4/10] method=standard, alpha=0.00, f1_macro=0.0351
[CV 5/10] method=standard, alpha=0.00, f1_macro=0.0541
[CV 6/10] method=standard, alpha=0.00, f1_macro=0.0185
[CV 7/10] method=standard, alpha=0.00, f1_macro=0.0566
[CV 8/10] method=standard, alpha=0.00, f1_macro=0.0000
[CV 9/10] method=standard, alpha=0.00, f1_macro=0.0185
[CV 10/10] method=standard, alpha=0.00, f1_macro=0.0556
f1_mean = 0.0418

[CV 1/10] method=standard, alpha=0.05, f1_macro=0.3243
[CV 2/10] method=standard, alpha=0.05, f1_macro=0.4850
[CV 3/10] method=standard, alpha=0.05, f1_macro=0.5392
[CV 4/10] method=standard, alpha=0.05, f1_macro=0.4324
[CV 5/10] method=standard, alpha=0.05, f1_macro=0.4591
[CV 6/10] method=standard, alpha=0.05, f1_macro=0.4050
[CV 7/10] method=standard, alpha=0.05, f1_macro=0.2302
[CV 8/10] method=standard, alpha=0.05, f1_macr

[CV 6/10] method=standard, alpha=0.70, f1_macro=0.1500
[CV 7/10] method=standard, alpha=0.70, f1_macro=0.1333
[CV 8/10] method=standard, alpha=0.70, f1_macro=0.0884
[CV 9/10] method=standard, alpha=0.70, f1_macro=0.2184
[CV 10/10] method=standard, alpha=0.70, f1_macro=0.1389
f1_mean = 0.2239

[CV 1/10] method=standard, alpha=0.75, f1_macro=0.2617
[CV 2/10] method=standard, alpha=0.75, f1_macro=0.2289
[CV 3/10] method=standard, alpha=0.75, f1_macro=0.1622
[CV 4/10] method=standard, alpha=0.75, f1_macro=0.2032
[CV 5/10] method=standard, alpha=0.75, f1_macro=0.2576
[CV 6/10] method=standard, alpha=0.75, f1_macro=0.1500
[CV 7/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 8/10] method=standard, alpha=0.75, f1_macro=0.0896
[CV 9/10] method=standard, alpha=0.75, f1_macro=0.1018
[CV 10/10] method=standard, alpha=0.75, f1_macro=0.0952
f1_mean = 0.1550

[CV 1/10] method=standard, alpha=0.80, f1_macro=0.2617
[CV 2/10] method=standard, alpha=0.80, f1_macro=0.2289
[CV 3/10] method=standard, 

[CV 1/10] method=standard-support, alpha=0.35, f1_macro=0.1950
[CV 2/10] method=standard-support, alpha=0.35, f1_macro=0.1886
[CV 3/10] method=standard-support, alpha=0.35, f1_macro=0.1950
[CV 4/10] method=standard-support, alpha=0.35, f1_macro=0.1345
[CV 5/10] method=standard-support, alpha=0.35, f1_macro=0.1597
[CV 6/10] method=standard-support, alpha=0.35, f1_macro=0.2382
[CV 7/10] method=standard-support, alpha=0.35, f1_macro=0.1938
[CV 8/10] method=standard-support, alpha=0.35, f1_macro=0.1641
[CV 9/10] method=standard-support, alpha=0.35, f1_macro=0.2198
[CV 10/10] method=standard-support, alpha=0.35, f1_macro=0.1628
f1_mean = 0.1852

[CV 1/10] method=standard-support, alpha=0.40, f1_macro=0.1055
[CV 2/10] method=standard-support, alpha=0.40, f1_macro=0.1523
[CV 3/10] method=standard-support, alpha=0.40, f1_macro=0.1597
[CV 4/10] method=standard-support, alpha=0.40, f1_macro=0.1597
[CV 5/10] method=standard-support, alpha=0.40, f1_macro=0.1055
[CV 6/10] method=standard-support, a

[CV 8/10] method=standard-support, alpha=0.95, f1_macro=0.1084
[CV 9/10] method=standard-support, alpha=0.95, f1_macro=0.2125
[CV 10/10] method=standard-support, alpha=0.95, f1_macro=0.1329
f1_mean = 0.1449

[CV 1/10] method=standard-support, alpha=1.00, f1_macro=0.1055
[CV 2/10] method=standard-support, alpha=1.00, f1_macro=0.1523
[CV 3/10] method=standard-support, alpha=1.00, f1_macro=0.2139
[CV 4/10] method=standard-support, alpha=1.00, f1_macro=0.1886
[CV 5/10] method=standard-support, alpha=1.00, f1_macro=0.1597
[CV 6/10] method=standard-support, alpha=1.00, f1_macro=0.1597
[CV 7/10] method=standard-support, alpha=1.00, f1_macro=0.2198
[CV 8/10] method=standard-support, alpha=1.00, f1_macro=0.2198
[CV 9/10] method=standard-support, alpha=1.00, f1_macro=0.1429
[CV 10/10] method=standard-support, alpha=1.00, f1_macro=0.1084
f1_mean = 0.1671

[CV 1/10] method=ratio-support, alpha=0.00, f1_macro=0.1622
[CV 2/10] method=ratio-support, alpha=0.00, f1_macro=0.1345
[CV 3/10] method=ratio-

[CV 10/10] method=ratio-support, alpha=0.55, f1_macro=0.1000
f1_mean = 0.1684

[CV 1/10] method=ratio-support, alpha=0.60, f1_macro=0.1886
[CV 2/10] method=ratio-support, alpha=0.60, f1_macro=0.1345
[CV 3/10] method=ratio-support, alpha=0.60, f1_macro=0.1741
[CV 4/10] method=ratio-support, alpha=0.60, f1_macro=0.1886
[CV 5/10] method=ratio-support, alpha=0.60, f1_macro=0.1622
[CV 6/10] method=ratio-support, alpha=0.60, f1_macro=0.2382
[CV 7/10] method=ratio-support, alpha=0.60, f1_macro=0.2198
[CV 8/10] method=ratio-support, alpha=0.60, f1_macro=0.1111
[CV 9/10] method=ratio-support, alpha=0.60, f1_macro=0.1667
[CV 10/10] method=ratio-support, alpha=0.60, f1_macro=0.1000
f1_mean = 0.1684

[CV 1/10] method=ratio-support, alpha=0.65, f1_macro=0.1886
[CV 2/10] method=ratio-support, alpha=0.65, f1_macro=0.1345
[CV 3/10] method=ratio-support, alpha=0.65, f1_macro=0.1741
[CV 4/10] method=ratio-support, alpha=0.65, f1_macro=0.1886
[CV 5/10] method=ratio-support, alpha=0.65, f1_macro=0.1622
[C

In [118]:
f1 = [0 for i in range(n)]
accuracy = [0 for i in range(n)]
f1_binary = [0 for i in range(n)]

for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
    bin_cls = fcalc.classifier.PatternBinaryClassifier(
        X.iloc[train_index].values, 
        y.iloc[train_index].to_numpy(),
        method=method_best_p,
        alpha=alpha_best_p
    )
    bin_cls.predict(X.iloc[test_index].values)
    
    accuracy[i], f1[i], f1_binary[i] = results_fca(y.iloc[test_index], bin_cls.predictions)
    
    print(f"[CV {i+1}/{n}] method={method_best_p}, alpha={alpha_best_p:0.2f}", 
          f"accuracy={accuracy[i]:0.4f}, f1_macro={f1[i]:0.4f}, f1_binary={f1_binary[i]:0.4f}")

f1_mean = np.mean(f1)
accuracy_mean = np.mean(accuracy)
f1_binary_mean = np.mean(f1_binary)

print(f"\n accuracy={accuracy_mean:0.4f}, f1_macro={f1_mean:0.4f}, f1_binary={f1_binary_mean:0.4f}")

[CV 1/10] method=standard, alpha=0.10 accuracy=0.4595, f1_macro=0.4591, f1_binary=0.4737
[CV 2/10] method=standard, alpha=0.10 accuracy=0.4865, f1_macro=0.4804, f1_binary=0.5366
[CV 3/10] method=standard, alpha=0.10 accuracy=0.5405, f1_macro=0.5281, f1_binary=0.6047
[CV 4/10] method=standard, alpha=0.10 accuracy=0.4865, f1_macro=0.4850, f1_binary=0.5128
[CV 5/10] method=standard, alpha=0.10 accuracy=0.3784, f1_macro=0.3766, f1_binary=0.3429
[CV 6/10] method=standard, alpha=0.10 accuracy=0.4324, f1_macro=0.4308, f1_binary=0.4000
[CV 7/10] method=standard, alpha=0.10 accuracy=0.3611, f1_macro=0.2517, f1_binary=0.3030
[CV 8/10] method=standard, alpha=0.10 accuracy=0.4444, f1_macro=0.3172, f1_binary=0.4118
[CV 9/10] method=standard, alpha=0.10 accuracy=0.4444, f1_macro=0.3140, f1_binary=0.4444
[CV 10/10] method=standard, alpha=0.10 accuracy=0.3333, f1_macro=0.2319, f1_binary=0.3333

 accuracy=0.4367, f1_macro=0.3875, f1_binary=0.4363
